In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr


In [2]:
ibm = pdr.get_data_yahoo("IBM", start="2017-01-01", end="2021-12-31")
ibm

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-03,167.869995,166.009995,167.000000,167.190002,2934300.0,134.699097
2017-01-04,169.869995,167.360001,167.770004,169.259995,3381400.0,136.366791
2017-01-05,169.389999,167.259995,169.250000,168.699997,2682300.0,135.915634
2017-01-06,169.919998,167.520004,168.690002,169.529999,2945500.0,136.584351
2017-01-09,169.800003,167.619995,169.470001,167.649994,3189900.0,135.069656
...,...,...,...,...,...,...
2021-10-25,128.649994,126.940002,127.529999,127.639999,6374000.0,127.639999
2021-10-26,128.300003,126.760002,127.519997,127.129997,8520900.0,127.129997
2021-10-27,127.879997,125.010002,127.440002,125.169998,6973400.0,125.169998
